In [1]:
import glob
import pandas as pd
import re
import ast

from src.experiments.experiment import *

In [2]:
reg_df = read_result_csv('reg')
class_df = read_result_csv('class')

reg_avg = average_reg_metrics(reg_df)
class_avg = average_class_metrics(class_df)

reg_df_extract = extract_reg(reg_df)
class_df_extract = extract_class(class_df)

In [3]:
reg_avg.head()


,model_name,train_time,pred_time,norm_mse,r2
0,DecisionTreeRegressor,0.4844 ± 0.77,0.0004 ± 0.0,0.7269 ± 0.45,0.2591 ± 0.46
1,ForestBasedTree,41.6762 ± 42.44,0.0849 ± 0.09,0.6945 ± 0.4,0.2984 ± 0.39
2,RandomForestRegressor,1.1142 ± 2.1,0.0013 ± 0.0,0.5111 ± 0.34,0.4818 ± 0.34


In [4]:
class_avg.head()

,model_name,train_time,pred_time,accuracy
0,DecisionTreeClassifier,2.2256 ± 0.83,0.0006 ± 0.0,0.4899 ± 0.16
1,ForestBasedTree,131.5652 ± 100.05,0.1411 ± 0.16,0.3054 ± 0.14
2,PrevPaperClassifier,22942.2068 ± 21754.72,28.7225 ± 26.08,0.2948 ± 0.15
3,RandomForestClassifier,0.193 ± 0.14,0.0017 ± 0.0,0.5616 ± 0.16
